## Ingestion de la carpeta JSON multi-linhas "production_country"

#### Paso 0 - Actualización de variables y funciones y validaciones

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
#Chama a função para validar se o arquivo a ser processado existe
valida_pasta_arquivo(bronze_folder_path, v_file_date, "production_country", "production_country_*.json")

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
#%fs
#ls dbfs:/mnt/moviehistoryadilmor/bronze/production_country/

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
#Definindo o schema na variavel xxxx_schema
productions_countries_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("countryId", IntegerType(), True)
])

In [0]:
productions_countries_df = spark.read \
               .schema(productions_countries_schema) \
               .option("multiLine", True) \
               .json(f"{bronze_folder_path}/{v_file_date}/production_country/production_country_*.json")

In [0]:
#display(productions_countries_df)
#productions_countries_df.count()

#### Paso 2 - Renombrar el nombre de las columnas,  añadir "ingestion_date" y "enviroment" Y dropar las columnas no deseadas

In [0]:
from pyspark.sql.functions import current_timestamp, lit, col

In [0]:
productions_countries_final_df = add_ingestion_date(productions_countries_df) \
                    .withColumnsRenamed({"movieId": "movie_id", 
                                         "countryId": "country_id"}) \
                    .withColumn("enviroment", lit("production")) \
                    .withColumn("file_date", lit(v_file_date))


In [0]:
#display(productions_countries_final_df)

#### Paso 3 - Escribir datos en el datalake en formato parquet

In [0]:
#Elimina partição se ela já existir
#drop_partition_if_exists(productions_countries_final_df, "movie_silver", "productions_countries", "file_date")

In [0]:
#productions_countries_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/productions_countries/")
# productions_countries_final_df.write \
                # .mode("append") \
                # .partitionBy("file_date") \
                # .format("parquet") \
                # .saveAsTable("movie_silver.productions_countries")

#Caso a tabela não existir, cria a tabela com os dados, senão efetua um merge dos dados na tabela
merge_delta_lake(productions_countries_final_df, "movie_silver", "productions_countries", silver_folder_path, "tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id", "file_date")

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/silver/productions_countries

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/productions_countries"))
#(spark.read.parquet("/mnt/moviehistoryadilmor/silver/productions_countries")).count()
#spark.sql("select * from movie_silver.productions_countries").display() 
display(spark.sql("SELECT file_date, \
                          COUNT(1) \
                   FROM movie_silver.productions_countries \
                   GROUP BY file_date"
                   )
        )

In [0]:
dbutils.notebook.exit("Success")